# Capstone project week 5

In [26]:
import pandas as pd
data = pd.read_html('https://en.wikipedia.org/wiki/Planning_Areas_of_Singapore',skiprows=1)
df = data[1]
df.columns=['Neighborhood','Malay', 'Chinese','Pinyin','Tamil','Region','Area','Population','Density']
df.head()

,Neighborhood,Malay,Chinese,Pinyin,Tamil,Region,Area,Population,Density
0,Ang Mo Kio,NaN,宏茂桥,Hóng mào qiáo,ஆங் மோ கியோ,North-East,13.94,174770.0,12538.2
1,Bedok,*,勿洛,Wù luò,பிடோ,East,21.69,289750.0,13360.5
2,Bishan,NaN,碧山,Bì shān,பீஷான்,Central,7.62,90700.0,11896.6
3,Boon Lay,NaN,文礼,Wén lǐ,பூன் லே,West,8.23,30.0,3.6
4,Bukit Batok,*,武吉巴督,Wǔjí bā dū,புக்கிட் பாத்தோக்,West,11.13,139270.0,12513.0


In [27]:
select_columns = ['Neighborhood','Region','Area','Population','Density']
df = df[select_columns]
df = df[df.Region == 'East']
df

,Neighborhood,Region,Area,Population,Density
1,Bedok,East,21.69,289750.0,13360.5
9,Changi,East,40.61,2530.0,62.3
10,Changi Bay,East,1.70,NaN,NaN
30,Pasir Ris,East,15.02,139890.0,9313.0
31,Paya Lebar,East,11.69,40.0,3.4
46,Tampines,East,20.89,261230.0,12506.2


Because the Geocoder service is currently not available and since the number of records is 6, I have queried the longitude and latitude on Google Map. The values will be added to the data frame.

In [28]:
latitudes = [1.323608, 1.3450101, 1.3219708, 1.3720937, 1.3516087, 1.3495907]
longitudes = [103.9273405, 103.9832089, 104.0290022, 103.9473728, 103.8995160, 103.9567879]
df['Latitude'] = pd.Series(latitudes, index=df.index)
df['Longitude'] = pd.Series(longitudes, index=df.index)
df.head()

,Neighborhood,Region,Area,Population,Density,Latitude,Longitude
1,Bedok,East,21.69,289750.0,13360.5,1.323608,103.927340
9,Changi,East,40.61,2530.0,62.3,1.345010,103.983209
10,Changi Bay,East,1.70,NaN,NaN,1.321971,104.029002
30,Pasir Ris,East,15.02,139890.0,9313.0,1.372094,103.947373
31,Paya Lebar,East,11.69,40.0,3.4,1.351609,103.899516


In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [33]:
CLIENT_ID = 'F4W32SODNFMMF41X5J1VSEEBIQKLBTWVC4F4ELCU2SGHXP4B' # your Foursquare ID
CLIENT_SECRET = 'HR0PR30WLD2JSOKKPB3TLMGE3OZZSOF5LVPH3GDJIAHS512E' # your Foursquare Secret
VERSION = '20190126' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: F4W32SODNFMMF41X5J1VSEEBIQKLBTWVC4F4ELCU2SGHXP4B
CLIENT_SECRET:HR0PR30WLD2JSOKKPB3TLMGE3OZZSOF5LVPH3GDJIAHS512E


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
singapore_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Bedok
Changi
Changi Bay
Pasir Ris
Paya Lebar
Tampines


In [36]:
singapore_venues.shape

(119, 7)

In [38]:
singapore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bedok,1.323608,103.92734,Ya Kun Kaya Toast 亞坤,1.324095,103.929198,Coffee Shop
1,Bedok,1.323608,103.92734,FairPrice Fínest,1.324140,103.929260,Supermarket
2,Bedok,1.323608,103.92734,Bedok Chwee Kueh 勿洛水粿,1.324903,103.930250,Chinese Restaurant
3,Bedok,1.323608,103.92734,Baker And Cook,1.320861,103.924345,Bakery
4,Bedok,1.323608,103.92734,Din Tai Fung 鼎泰豐,1.324475,103.930164,Dumpling Restaurant


In [40]:
singapore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bedok,62,62,62,62,62,62
Changi,13,13,13,13,13,13
Changi Bay,5,5,5,5,5,5
Pasir Ris,29,29,29,29,29,29
Paya Lebar,3,3,3,3,3,3
Tampines,7,7,7,7,7,7


In [41]:
# one hot encoding
singapore_onehot = pd.get_dummies(singapore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
singapore_onehot['Neighborhood'] = singapore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [singapore_onehot.columns[-1]] + list(singapore_onehot.columns[:-1])
singapore_onehot = singapore_onehot[fixed_columns]

singapore_onehot.head()

,Neighborhood,Airport,Airport Service,Airport Terminal,American Restaurant,Asian Restaurant,Bakery,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Dumpling Restaurant,Fast Food Restaurant,Fishing Spot,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden Center,Gastropub,Gourmet Shop,Grocery Store,Hong Kong Restaurant,Hotpot Restaurant,Housing Development,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lighthouse,Malay Restaurant,Men's Store,Military Base,Movie Theater,Noodle House,Pizza Place,Plaza,Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Soup Place,South Indian Restaurant,Sporting Goods Shop,Supermarket,Sushi Restaurant,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wings Joint
0,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Bedok,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Bedok,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [42]:
singapore_grouped = singapore_onehot.groupby('Neighborhood').mean().reset_index()
singapore_grouped.head()

,Neighborhood,Airport,Airport Service,Airport Terminal,American Restaurant,Asian Restaurant,Bakery,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Dumpling Restaurant,Fast Food Restaurant,Fishing Spot,Food Court,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Garden Center,Gastropub,Gourmet Shop,Grocery Store,Hong Kong Restaurant,Hotpot Restaurant,Housing Development,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lighthouse,Malay Restaurant,Men's Store,Military Base,Movie Theater,Noodle House,Pizza Place,Plaza,Restaurant,Road,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Soup Place,South Indian Restaurant,Sporting Goods Shop,Supermarket,Sushi Restaurant,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Wings Joint
0,Bedok,0.000000,0.000000,0.000000,0.032258,0.016129,0.032258,0.000000,0.000000,0.0,0.016129,0.016129,0.016129,0.016129,0.016129,0.000000,0.000000,0.016129,0.048387,0.032258,0.016129,0.048387,0.000000,0.032258,0.016129,0.032258,0.000000,0.064516,0.016129,0.032258,0.016129,0.016129,0.016129,0.016129,0.0,0.016129,0.016129,0.0,0.032258,0.000000,0.032258,0.016129,0.016129,0.000000,0.032258,0.000000,0.000000,0.000000,0.016129,0.016129,0.016129,0.000000,0.000000,0.000000,0.016129,0.016129,0.016129,0.016129,0.0,0.000000,0.048387,0.048387,0.016129,0.000000,0.016129,0.016129
1,Changi,0.076923,0.076923,0.153846,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.153846,0.000000,0.076923,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.076923,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000
2,Changi Bay,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Pasir Ris,0.000000,0.000000,0.000000,0.000000,0.034483,0.068966,0.034483,0.034483,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.068966,0.034483,0.068966,0.000000,0.000000,0.068966,0.034483,0.000000,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.034483,0.000000,0.0,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.034483,0.068966,0.034483,0.034483,0.000000,0.0,0.000000,0.034483,0.034483,0.034483,0.000000,0.000000,0.000000
4,Paya Lebar,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
singapore_grouped.shape

(6, 66)

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = singapore_grouped['Neighborhood']

for ind in np.arange(singapore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bedok,Food Court,Coffee Shop,Sushi Restaurant,Supermarket,Café,Fried Chicken Joint,Dessert Shop,Malay Restaurant,Chinese Restaurant,Japanese Restaurant
1,Changi,Bus Station,Airport Terminal,Airport,Movie Theater,Café,Coffee Shop,Road,Men's Store,Sporting Goods Shop,Airport Service
2,Changi Bay,Boat or Ferry,Military Base,Wings Joint,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Dumpling Restaurant,Bus Stop
3,Pasir Ris,Café,Bus Station,Coffee Shop,Sandwich Place,Bakery,Fishing Spot,Italian Restaurant,Bus Line,Hong Kong Restaurant,Food Court
4,Paya Lebar,Bus Station,Lighthouse,Military Base,Bus Stop,Fried Chicken Joint,French Restaurant,Food Court,Fishing Spot,Fast Food Restaurant,Dumpling Restaurant


Set the number of clusters to 3 to cluster the neighbourhoods into 3 groups.

In [49]:
# set number of clusters
kclusters = 3

singapore_grouped_clustering = singapore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 2, 1, 0, 1], dtype=int32)

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [51]:
singapore_merged = df

# merge singapore_grouped with singapore_data to add latitude/longitude for each neighborhood
singapore_merged = singapore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

singapore_merged.head() # check the last columns!

,Neighborhood,Region,Area,Population,Density,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bedok,East,21.69,289750.0,13360.5,1.323608,103.927340,1,Food Court,Coffee Shop,Sushi Restaurant,Supermarket,Café,Fried Chicken Joint,Dessert Shop,Malay Restaurant,Chinese Restaurant,Japanese Restaurant
9,Changi,East,40.61,2530.0,62.3,1.345010,103.983209,1,Bus Station,Airport Terminal,Airport,Movie Theater,Café,Coffee Shop,Road,Men's Store,Sporting Goods Shop,Airport Service
10,Changi Bay,East,1.70,NaN,NaN,1.321971,104.029002,2,Boat or Ferry,Military Base,Wings Joint,Chinese Restaurant,Clothing Store,Coffee Shop,Cosmetics Shop,Dessert Shop,Dumpling Restaurant,Bus Stop
30,Pasir Ris,East,15.02,139890.0,9313.0,1.372094,103.947373,1,Café,Bus Station,Coffee Shop,Sandwich Place,Bakery,Fishing Spot,Italian Restaurant,Bus Line,Hong Kong Restaurant,Food Court
31,Paya Lebar,East,11.69,40.0,3.4,1.351609,103.899516,0,Bus Station,Lighthouse,Military Base,Bus Stop,Fried Chicken Joint,French Restaurant,Food Court,Fishing Spot,Fast Food Restaurant,Dumpling Restaurant


In [62]:
# create map
latitude = 1.345010
longitude = 103.927340
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_merged['Latitude'], singapore_merged['Longitude'], singapore_merged['Neighborhood'], singapore_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters